# Update DB 
`get_additional_data` must be run prior (OR retrieve CSVs from google drive <a href="https://drive.google.com/drive/folders/1s_6b-szIrBIscnAwA6HV__kgFDCoOlpd?usp=sharing">here</a>)

This file uses CSVs generated from get_additional_data (descriptions, trailers, posters) and get_service_provider (service provider links) to update movie data in `imdb_movies` table. 

It may make sense to have `service_providers` as a seperate table and create a one to many relationship between the movie and their service providers. 

In [1]:
# imports 
import pandas as pd 
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv

In [5]:
# load .env file for keys
load_dotenv()

True

In [20]:
# create connection to prod DB
connection = psycopg2.connect(
                            user=os.getenv('DB_USER'),
                            password=os.getenv('DB_PASSWORD'),
                            host=os.getenv('HOST'),
                            port=os.getenv('PORT'),
                            database=os.getenv('DB_NAME')
                            )

## Add descriptions to movies

In [57]:
# read in description data
desc_df = pd.read_csv('../../data/description_results.csv', engine='python')
print(desc_df.shape)
desc_df.head()

(201352, 3)


,movie_id,tmdb_id,description
0,1051231,31223.0,In the Hands of the Gods is the true story of ...
1,1051244,573815.0,A group of talented youth exploited by the hea...
2,10513474,639651.0,A unique chance to explore Pier Paolo Pasolini...
3,10515086,599672.0,A meeting with a new inmate in the psychiatric...
4,10515340,531678.0,A strange disease is plaguing the city. Hoping...


In [58]:
# how many null values
desc_df['description'].isnull().sum()

27476

In [59]:
# any empty strings?
desc_df[desc_df['description'] == ""].shape[0]

0

In [60]:
# drop rows without a description
desc_df = desc_df.dropna(subset=['description'])
desc_df.shape

(173876, 3)

In [61]:
# fix movie_id
desc_df['movie_id'] = desc_df['movie_id'].astype(str)
def fill_id(id):
    """Adds leading zeroes back if necessary. This makes the id match the database."""
    if len(str(id)) < 7:
        length = len(str(id))
        id = "0"*(7 - length) + str(id)
    return str(id)

desc_df['movie_id'] = desc_df['movie_id'].apply(fill_id)

In [62]:
# update every movie we have a description for
cursor = connection.cursor()
update_query = "UPDATE movies SET description = %s WHERE movie_id = %s;"

for movie_id, desc in desc_df[['movie_id', 'description']].values:
    cursor.execute(update_query, (desc, movie_id))

connection.commit()
cursor.close()

In [80]:
# count every movie with a description
cursor = connection.cursor()

query = "SELECT COUNT(*) FROM movies WHERE description IS NOT NULL;"
cursor.execute(query)
desc_count = cursor.fetchone()

cursor.close()
desc_count

(173876,)

In [66]:
# get a test_id and desc
test_id = desc_df['movie_id'][0]
test_desc = desc_df['description'][0]
test_id, test_desc

('1051231',
 'In the Hands of the Gods is the true story of five young British freestyle footballers journey across the Americas to Argentina in the hope of meeting their hero, Diego Maradona. This coming-of-age road movie tells the story of a group of young men in pursuit of a lifelong dream.')

In [70]:
# ensure test_desc is same in DB
cursor = connection.cursor()
query = f"SELECT movie_id, description FROM movies WHERE movie_id = '{test_id}';"
cursor.execute(query)
print(cursor.fetchone())
cursor.close()

('1051231', 'In the Hands of the Gods is the true story of five young British freestyle footballers journey across the Americas to Argentina in the hope of meeting their hero, Diego Maradona. This coming-of-age road movie tells the story of a group of young men in pursuit of a lifelong dream.')


## Add trailer to movie

In [71]:
# read in trailer data 
trailer_df = pd.read_csv('../../data/trailer_data.csv')
print(trailer_df.shape)
trailer_df.head()

(50793, 4)


,movie_id,video_key,video_site,more_than_one
0,1051244,ztSS7hnEviY,YouTube,False
1,10515086,WA2NvFSHchk,YouTube,False
2,10515460,HQksgesFrFY,YouTube,False
3,10515480,QBNKpcUOWgI,YouTube,False
4,1051232,k9SdzYiyG14,YouTube,False


In [72]:
# any null values?
trailer_df.isnull().sum()

movie_id         0
video_key        0
video_site       0
more_than_one    0
dtype: int64

In [73]:
# what are the video_site values?
trailer_df['video_site'].value_counts()

YouTube    50205
Vimeo        588
Name: video_site, dtype: int64

In [74]:
# how many did have more than one on TMDb
trailer_df['more_than_one'].value_counts()

False    43481
True      7312
Name: more_than_one, dtype: int64

In [75]:
# fix movie_id
trailer_df['movie_id'] = trailer_df['movie_id'].astype(str)
trailer_df['movie_id'] = trailer_df['movie_id'].apply(fill_id)

In [81]:
# update every movie we have a trailer for
cursor = connection.cursor()
youtube_base = 'https://www.youtube.com/watch?v='
vimeo_base = 'https://vimeo.com/'

for movie_id, key, site in trailer_df[['movie_id', 'video_key', 'video_site']].values:
    if site == 'YouTube':
        trailer_url = youtube_base + str(key)
    else:
        trailer_url = vimeo_base + str(key)
    update_query = f"UPDATE movies SET trailer_url = '{trailer_url}' WHERE movie_id = '{movie_id}';"
    cursor.execute(update_query)

connection.commit()
cursor.close()

In [82]:
# count every movie with a trailer_url
cursor = connection.cursor()

query = "SELECT COUNT(*) from movies WHERE trailer_url IS NOT NULL;"
cursor.execute(query)
trailer_count = cursor.fetchone()

cursor.close()
trailer_count

(50793,)

In [89]:
# get test_id and key
test_id = trailer_df['movie_id'][1000]
test_key = trailer_df['video_key'][1000]
test_id, test_key

('1209379', 'AHne0pCugmw')

In [90]:
# ensure test_key is same in DB
cursor = connection.cursor()
query = f"SELECT movie_id, trailer_url FROM movies WHERE movie_id = '{test_id}';"
cursor.execute(query)
print(cursor.fetchone())
cursor.close()

('1209379', 'https://www.youtube.com/watch?v=AHne0pCugmw')


## Add poster to movies that didn't have one

In [11]:
# read in poster data 
poster_df = pd.read_csv('../../data/poster_data.csv')
print(poster_df.shape)
poster_df.head()

(201482, 3)


,movie_id,tmdb_id,poster_path
0,1051259,87397,/4c27iElQGoLSlrg2uZQUANHA4aa.jpg
1,1051834,533781,NaN
2,10520386,637881,/iASQXlZqZfERdwTnBxIRpfgKrsO.jpg
3,1052347,201899,NaN
4,10519798,676343,NaN


In [12]:
# how many null poster_paths?
poster_df['poster_path'].isnull().sum()

77522

In [13]:
# drop those rows
poster_df = poster_df.dropna(subset=['poster_path'])
poster_df.shape

(123960, 3)

In [16]:
# fix movie_id
poster_df['movie_id'] = poster_df['movie_id'].astype(str)
poster_df['movie_id'] = poster_df['movie_id'].apply(fill_id)

In [9]:
# count every movie with a poster_path before update
cursor = connection.cursor()

query = "SELECT COUNT(*) from movies WHERE poster_url IS NOT NULL;"
cursor.execute(query)
poster_count = cursor.fetchone()

cursor.close()
poster_count

(183087,)

In [17]:
# update every movie we have a poster for
cursor = connection.cursor()
base_query = "UPDATE movies SET poster_url = %s WHERE movie_id = %s;"

for movie_id, poster_path in poster_df[['movie_id', 'poster_path']].values:
    cursor.execute(base_query, (poster_path, movie_id))

connection.commit()
cursor.close()

In [21]:
# count every movie with a poster_path after update
cursor = connection.cursor()

query = "SELECT COUNT(*) from movies WHERE poster_url IS NOT NULL;"
cursor.execute(query)
poster_count = cursor.fetchone()

cursor.close()
poster_count

(194343,)

## Add service provider links to movies

In [4]:
provider_df = pd.read_csv('provider_data.csv')
print(provider_df.shape)
provider_df.head()

(224382, 7)


,movie_id,title,jw_id,jw_title,offer_provider_id,offer_urls,ratio
0,1051262,The Stalker Within,309961,The Evil Within,"7,7,7,7,192,3,3,192,3,192,9,9,10,10,10,10,3,19...",https://www.vudu.com/content/movies/details/Th...,73
1,1051263,Stolen Life,167298,Stolen Life,"25,10,10,9","https://www.fandor.com/films/stolen_life,https...",100
2,10513072,Don't Date Your Sister,434938,Don't Open Your Eyes,"10,10,10,10,2,2,2,2,12,9,9",https://www.amazon.com/gp/product/B07H4T4NQ2?c...,67
3,1051320,La cantatrice chauve,23259,La Bamba,"7,7,7,7,3,3,192,192,279,279,279,279,10,10,10,1...",https://www.vudu.com/content/movies/details/La...,36
4,10513286,Historia de mi nombre,403709,Marriage Story,"8,8,8","http://www.netflix.com/title/80223779,http://w...",40


We need to filter out rows with `ratio` values less than 90 because it is very likely it is not the correct movie if less than that.

In [5]:
filtered_df = provider_df[provider_df['ratio'] > 90]
print(filtered_df.shape)
filtered_df.head()

(29017, 7)


,movie_id,title,jw_id,jw_title,offer_provider_id,offer_urls,ratio
1,1051263,Stolen Life,167298,Stolen Life,"25,10,10,9","https://www.fandor.com/films/stolen_life,https...",100
9,10511068,Inheritance,311393,Inheritance,"3,3,7,7,7,7,192,192,3,3,192,192,9,9,10,10,10,10",https://play.google.com/store/movies/details/I...,100
11,10514932,What Do I Do Now?,659810,What Do I Do Now?,"9,9,10,10,10,10",https://www.amazon.com/gp/product/B075QVYP3H?c...,100
28,1051245,Moving Midway,78930,Moving Midway,"2,2,191,9,9",https://itunes.apple.com/us/movie/moving-midwa...,100
31,1051253,The Portal,319476,The Portal,"3,3,3,10,10,10,10,9,9,192,192,192,2,2,2,2,68,6...",https://play.google.com/store/movies/details/T...,100


`offer_provider_id` and `offer_urls` are string deliminated sequences. We can use the JustWatch API to get the provider details of each ID and the URLs will be saved to be linked on the movie detail page. There is other info we could've saved to better differentiate between the values with the same provider_id on the same movie. From what I saw it seems some are HD while some are SD, as well as some of the monetization types differ resulting in repeats of the same provider. 

We will probably want to filter out most service providers (other than Netflix, Amazon, and Hulu) because some are not popular enough to justify displaying theirs on the webpage. 

## New movies table info:
New table columns:
`movie_id
title_type
primary_title
original_title
is_adult
start_year
end_year
runtime_minutes
genres
poster_url
average_rating
num_votes
original_language
description
trailer_url`

New table schema:
`CREATE TABLE public.movies
(
    movie_id character varying COLLATE pg_catalog."default" NOT NULL,
    title_type character varying COLLATE pg_catalog."default",
    primary_title character varying COLLATE pg_catalog."default",
    original_title character varying COLLATE pg_catalog."default",
    is_adult boolean,
    start_year integer,
    end_year integer,
    runtime_minutes integer,
    genres character varying COLLATE pg_catalog."default",
    poster_url character varying COLLATE pg_catalog."default",
    average_rating real,
    num_votes integer,
    original_language character varying COLLATE pg_catalog."default",
    description character varying COLLATE pg_catalog."default",
    trailer_url character varying COLLATE pg_catalog."default",
    CONSTRAINT movies_pkey PRIMARY KEY (movie_id)
)`
